In [10]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import time
import pyproj
import glob
import pickle

In [11]:
airport_location = pd.read_csv('../../data/airport_positions_new.csv')

In [12]:
parameters = ['air_temperature_0m', 'air_temperature_2m', 'air_temperature_pl', 
              'relative_humidity_2m', 'precipitation_amount_acc', 'x_wind_10m', 'y_wind_10m', 
             'x_wind_pl', 'y_wind_pl', 'fog_area_fraction', 'surface_air_pressure', 'air_pressure_at_sea_level']

In [13]:
def transform_latitude_longitude_to_xy(latitude_values, longitude_values):
    # Create a pyproj CRS object
    proj4_str = '+proj=lcc +lat_0=63.3 +lon_0=15 +lat_1=63.3 +lat_2=63.3 +no_defs +R=6.371e+06'
    lcc_crs = pyproj.CRS.from_proj4(proj4_str)

    # Create a transformer for converting between lat/lon and x/y
    transformer = pyproj.Transformer.from_crs(pyproj.CRS("EPSG:4326"), lcc_crs, always_xy=True)

    # Transform lat/lon to x/y
    x_values, y_values = transformer.transform(longitude_values, latitude_values)

    return x_values, y_values

In [14]:
# Define folder with paths and prefix for files to read
folder_path = '/lustre/storeB/immutable/archive/projects/metproduction/meps/2022/'
file_prefix = 'meps_lagged_6_h_subset_2_5km_*.nc'
# file_pattern = os.path.join(folder_path, file_prefix)
#file_list = glob.glob(file_pattern)
output_folder = '/lustre/storeB/users/tonjek/msc/2024_msc_tonje_metar/00_data_preparation/data_extraction/2022/'

In [16]:
airport_identifier = airport_location['airport_identifier'].unique()
for month_folder in os.listdir(folder_path):
    month_path = os.path.join(folder_path, month_folder)
    all_timesteps_data = []
    for day_folder in os.listdir(month_path):
        day_path = os.path.join(month_path, day_folder)

        file_pattern = os.path.join(day_path, file_prefix)
        file_list = glob.glob(file_pattern)

        print(f"Processing day: Month: {month_folder}, Day: {day_folder}")

        for file_path in file_list:
            data = xr.open_mfdataset(file_path, chunks={'time':1})
            start_time = time.time()
            print(file_path)

            latitude_values = airport_location.latitude.values
            longitude_values = airport_location.longitude.values

            x_values, y_values = transform_latitude_longitude_to_xy(latitude_values, longitude_values)

            airport_six_timesteps = []

            for airport_idx in range(len(latitude_values)):
                nearest_x = x_values[airport_idx]
                nearest_y = y_values[airport_idx]

                interpolated_data = data[parameters].interp(x=float(nearest_x), y=float(nearest_y))
                
                x_min = float(interpolated_data['x'].values - (10 * 2500))
                x_max = float(interpolated_data['x'].values + (11 * 2500))
                y_min = float(interpolated_data['y'].values - (10 * 2500))
                y_max = float(interpolated_data['y'].values + (11 * 2500))

                start_datetime = pd.to_datetime(data.time.values[0])
                end_datetime = start_datetime + pd.DateOffset(hours=5)

                extracted_data = data[parameters].sel(
                    ensemble_member=0,
                    time=slice(start_datetime, end_datetime),
                    x=slice(x_min, x_max),
                    y=slice(y_min, y_max),
                    height0=0,
                    height1=2,
                    height2 = 10,
                    height_above_msl=0,
                    
                )
                
                pressure_levels = [700, 850, 925]
                for pressure_level in pressure_levels:
                    var_temp = f'air_temperature_pl_{pressure_level}'
                    var_x_wind = f'x_wind_pl_{pressure_level}'
                    var_y_wind = f'y_wind_pl_{pressure_level}'
                    extracted_data[var_temp] = extracted_data.sel(pressure=pressure_level)['air_temperature_pl']
                    extracted_data[var_x_wind] = extracted_data.sel(pressure=pressure_level)['x_wind_pl']
                    extracted_data[var_y_wind] = extracted_data.sel(pressure=pressure_level)['y_wind_pl']
    
                extracted_data = extracted_data.drop_vars(['air_temperature_pl', 'x_wind_pl', 'y_wind_pl', 'pressure'])
                
                # Add new coords for the extracted x and y values, and drop the original ones
                extracted_data.coords['x_point'] = (('x',), extracted_data.x.values)
                extracted_data.coords['y_point'] = (('y',), extracted_data.y.values)
                extracted_data = extracted_data.drop_vars(['x', 'y'])
                
                # Calculate hourly precipitation amount
                #hourly_precipitation = extracted_data['precipitation_amount_acc'].diff(dim='time', label='lower')

                # Replace the first value in the diff with the original value 
                #hourly_precipitation[0, :, :] = extracted_data['precipitation_amount_acc'][0, :, :]

                # Set negative differences to 0
                #hourly_precipitation = xr.where(hourly_precipitation < 0, 0, hourly_precipitation)

                #extracted_data['precipitation_amount_calculated'] = hourly_precipitation

                # Convert Kelvin to Celsius in temperature columns
                extracted_data['air_temperature_0m'] = extracted_data['air_temperature_0m'] - 273.15
                extracted_data['air_temperature_2m'] = extracted_data['air_temperature_2m'] - 273.15
                extracted_data['air_temperature_pl_700'] = extracted_data['air_temperature_pl_700'] - 273.15
                extracted_data['air_temperature_pl_850'] = extracted_data['air_temperature_pl_850'] - 273.15
                extracted_data['air_temperature_pl_925'] = extracted_data['air_temperature_pl_925'] - 273.15


                # Add airport as a new dimension
                extracted_data = extracted_data.expand_dims({'airport': [airport_identifier[airport_idx]]})                
                
                airport_six_timesteps.append(extracted_data)
            combined_data = xr.concat(airport_six_timesteps, dim='airport')
            all_timesteps_data.append(combined_data)
            end_time = time.time()
            print(f'Time taken to extract info from {file_path}: {end_time - start_time}')
            #print(f'Dimensions after concatenation for day {day_folder}: {combined_data.dims}')
    final_combined_data = xr.concat(all_timesteps_data, dim='time')
    final_combined_data = final_combined_data.drop_vars('ensemble_member')

    # Save pickle
    pickle_filename = f'2022_{month_folder}_grid.pkl'
    pickle_filepath = os.path.join(output_folder, pickle_filename)

    with open(pickle_filepath, 'wb') as pickle_file:
        pickle.dump(final_combined_data, pickle_file)

    print(f"Saved pickle file for file {file_path}: {pickle_filepath}")

Processing day: Month: 11, Day: 19
/lustre/storeB/immutable/archive/projects/metproduction/meps/2022/11/19/meps_lagged_6_h_subset_2_5km_20221119T06Z.nc


KeyboardInterrupt: 